In [40]:
import os
import requests
import configparser
import flask,pickle,sklearn,json
import pandas as pd

In [41]:
os.getcwd()

'C:\\Users\\meet9\\Downloads\\Deployment Practise\\Project\\GiveMeSomeCredit\\Scripts'

In [42]:
#os.chdir(r'..\\')

In [43]:
config = configparser.ConfigParser()
config.read(r'../Config/config.ini')

['../Config/config.ini']

In [44]:
json.loads(config.get("Variables","var_list"))

['RevolvingUtilizationOfUnsecuredLines',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfTimes90DaysLate',
 'age',
 'MonthlyIncome',
 'DebtRatio',
 'NumberOfDependents',
 'NumberRealEstateLoansOrLines',
 'NumberOfOpenCreditLinesAndLoans']

In [45]:
config['Output_message']['pos']

'The user is not going to be default and will continue the service'

In [ ]:
input_val = {'RevolvingUtilizationOfUnsecuredLines':0.5, 
             'NumberOfTime30-59DaysPastDueNotWorse':1,
             'NumberOfTime60-89DaysPastDueNotWorse':0,
             'NumberOfTimes90DaysLate':0,
             'age':3,
             'MonthlyIncome':10120.0, 
             'DebtRatio':0.2982, 
             'NumberOfDependents':1.0,
             'NumberRealEstateLoansOrLines':2, 
             'NumberOfOpenCreditLinesAndLoans':15}

In [ ]:
dt = pd.DataFrame(input_val,index=range(1))
dt

In [ ]:
#url = 'http://192.68.31.26:8080/predict'

In [ ]:
#response = requests.post(url, json=input_val)

In [ ]:
#response

In [ ]:
#print(response.json()['Result'])

In [ ]:
arr = [[0.05,1,1,2,30,23000,0.5,5,1,1]]

In [ ]:
data = pd.DataFrame(arr,columns = config['Variables']['var_list'])

In [ ]:
data

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from data_transform import *

In [ ]:
class Data_Preparation(BaseEstimator,TransformerMixin):
    def __init__(self):        
        self.interval = (0,18, 25, 35, 60, 110)
        self.cats = [0,1,2,3,4]    
        
    def fit(self,X):
        return self
    
    def transform(self,X):
        X=X.copy()
        
        Delay_param = ["NumberOfTime30-59DaysPastDueNotWorse",
               "NumberOfTime60-89DaysPastDueNotWorse",
               "NumberOfTimes90DaysLate"]
        
        X['Weighted_Delay_sum'] = X[Delay_param[0]].values[0]*0.1+\
                                 X[Delay_param[1]].values[0]*0.3+\
                                 X[Delay_param[2]].values[0]*0.6
        
        X['age_cat'] = pd.cut([X.age.values[0]], self.interval, labels=self.cats)
        
        X['NumberOfDependentsTR'] = X['NumberOfDependents']
        X.loc[X.NumberOfDependentsTR>=7,'NumberOfDependentsTR'] = 7

        X['NumberRealEstateLoansOrLinesTR'] = X['NumberRealEstateLoansOrLines']
        X.loc[X.NumberRealEstateLoansOrLinesTR>=11,'NumberRealEstateLoansOrLinesTR'] = 11

        X['NumberOfOpenCreditLinesAndLoansTR'] = X['NumberOfOpenCreditLinesAndLoans']
        X.loc[X.NumberOfOpenCreditLinesAndLoansTR>=25,'NumberOfOpenCreditLinesAndLoansTR'] = 25
        
        transformed_feature = ['RevolvingUtilizationOfUnsecuredLines',
                           'Weighted_Delay_sum', 
                           'age_cat',
                           'MonthlyIncome', 
                           'DebtRatio', 
                           'NumberOfDependentsTR',
                           'NumberRealEstateLoansOrLinesTR', 
                           'NumberOfOpenCreditLinesAndLoansTR']
                
        return X[transformed_feature]

In [ ]:
dc = Data_Preparation()

In [ ]:
dc.fit_transform(data)